# Load Data
- 下載科技股歷史紀錄

In [ ]:
import yfinance
import pandas as pd
import numpy as np
import math

# Stock
import requests
import yfinance as yf
'''link = 'https://quality.data.gov.tw/dq_download_json.php?nid=11549&md5_url=bb878d47ffbe7b83bfc1b41d0b24946e'
all_stock = requests.get(link)
all_stock = pd.DataFrame(all_stock.json())
stockid_list = []
for i in all_stock['證券代號']:
    stockid = float(i[:4])
    if ((stockid>=2300 and stockid<2500) or 
        (stockid>=3000 and stockid<3100) or stockid==6128):
        stockid_list.append(i+".TW")'''

stockid_list = [i for i in range(2300, 2501)]
[stockid_list.append(i) for i in range(3000, 3101)]


stocks = pd.DataFrame()
stocks['證券代號'] = stockid_list
# stocks.reindex
stocks['證券歷史'] = None

allstock_hist = []
n = 0
for k, i in enumerate(stockid_list):
    try:
        i = str(i) + '.TW'
        hist = yf.download(i)
        stocks['證券歷史'][k] = hist
    except:
        stocks['證券歷史'][k] = None



# Store Data

In [ ]:
import pickle

with open('data/stock_allhistory', 'wb') as fi:
    pickle.dump(stocks, fi)


# Load Data from Pickle

In [ ]:
import pickle
import os

def get_stockhistdata(filepath):
    if os.path.getsize(filepath) > 0:
        with open(filepath, "rb") as f:
            unpicker = pickle.Unpickler(f)
            scores = unpicker.load()
    else:
        return 'bad'
    return scores

filepath = 'Data/stock_history'
hist_data = get_stockhistdata('Data/stock_history')
hist_data

In [ ]:
import os

if os.path.getsize('data/stock_history') > 0:  
  
    with open('data/stock_history', "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        scores = unpickler.load()
scores

# Select Target Stock and Categorical label
- 標記買進賣出訊號

In [ ]:
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import os 
import pandas as pd
import numpy as np
import math

# 預測20天後股價上漲或下跌
days_lag = -5
label = []
for i in (hist_data['證券歷史'][1]['Close'].shift(days_lag)/
    hist_data['證券歷史'][1]['Close']):
    if i>1.00:
        label.append(1)
    elif i <1.00:
        label.append(-1)
    else:
        label.append(0)

stock_categorical = pd.DataFrame(hist_data['證券歷史'][1])
stock_categorical['label'] = label

stock_categorical['Close'][5000:-6].plot()
stock_categorical['label'][5000:].plot(secondary_y = True)
stock_categorical

# Preprocessing

In [181]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torchvision
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

### Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1))
scaler.fit_transform(stock_categorical, y=1)

### Windows

In [183]:
win = 2
data_X = []
data_y = []
# 去掉最後days_lag的比數，因為沒資料加入0
for i in range(win, (len(stock_categorical)+days_lag)):
    # print(stock_categorical.iloc[i:i+win, :6])
    data_X.append(stock_categorical.iloc[i-win:i, :])
    data_y.append(stock_categorical.iloc[i, 6])


### Train Test Split
- Test 不測試準確度，用在回測部分

In [184]:
train_X = np.array(data_X[:int(len(data_X)*0.8)])
train_y = np.array(data_y[:int(len(data_y)*0.8)])
test_X = np.array(data_X[int(len(data_X)*0.8):])
test_y = np.array(data_y[int(len(data_y)*0.8):])
train_y.shape

(4532,)

### Encoding: Onehot

In [ ]:
class MyDataset(data.Dataset):
    def __init__(self, X, y):

        self.X = X
        self.y = y

    def __getitem__(self, index):#返回的是tensor
        X_, y_ = self.X[index], self.y[index]
        return X_, y_

    def __len__(self):
        return len(self.X)

from sklearn.preprocessing import OneHotEncoder
train_X = torch.tensor(train_X, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)
onehotencoder = OneHotEncoder()
train_yencoding = onehotencoder.fit_transform(train_y.reshape(len(train_y), 1)).toarray()

test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

train_loader = torch.utils.data.DataLoader(
    MyDataset(train_X, train_yencoding), batch_size = 300, shuffle=False, drop_last = True)

val_loader = torch.utils.data.DataLoader(
    MyDataset(test_X, test_y), batch_size = 300, shuffle=False, drop_last = True)

### Encoding: Label

In [ ]:
class MyDataset(data.Dataset):
    def __init__(self, X, y):

        self.X = X
        self.y = y

    def __getitem__(self, index):#返回的是tensor
        X_, y_ = self.X[index], self.y[index]
        return X_, y_

    def __len__(self):
        return len(self.X)

# from sklearn import preprocessing
train_X = torch.tensor(train_X, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)
# le = preprocessing.LabelEncoder()
# le.fit(train_y)

test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

train_loader = torch.utils.data.DataLoader(
    MyDataset(train_X, train_y), batch_size = 300, shuffle=False, drop_last = True)

val_loader = torch.utils.data.DataLoader(
    MyDataset(test_X, test_y), batch_size = 300, shuffle=False, drop_last = True)

# Model: Pytorch

In [187]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim# Hidden dimensions
        self.num_layers = num_layers# Number of hidden layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)# (batch_dim, seq_dim, feature_dim)
       
        self.fc = nn.Linear(hidden_dim, output_dim)# Readout layer

    def forward(self, i):
        
        h0 = torch.zeros([self.num_layers, i.shape[0], self.hidden_dim], dtype=torch.double) #.requires_grad_()
        c0 = torch.zeros([self.num_layers, i.shape[0], self.hidden_dim], dtype=torch.double) #.requires_grad_()
        
        out, _ = self.lstm.forward(i, (h0.detach().float(), c0.detach().float()))  # output shape (batch, sequence, hidden_dim)
        out = self.fc(out[:, -1, :])  
        out_softmax = F.softmax(out)
        # print(out_softmax)
        # print('---------------')
        # print(out_softmax)
        
        return out

### with Onehot Encoding

In [ ]:
model = LSTM(input_dim=7, hidden_dim=10, num_layers=2, output_dim=2)
model.train()
optimizer = optim.Adadelta(model.parameters(), lr = 5)
criterion = torch.nn.CrossEntropyLoss()
epochs = 50
for epoch in range(epochs):
    correct = 0
    all = 0
    for _, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        # print(y)
        pred_y = model(X)
        #print(pred_y.shape)
        # print(pred_y[1])
        #print(pred_y[1]*(-1))
        loss = criterion(y, pred_y)
        loss.backward()
        optimizer.step()
        print(f'-----{epoch}------')
        for i in range(len(pred_y)):
            if np.argmax(pred_y[i].detach().numpy()) == np.argmax(y[i].detach().numpy()):           
                correct+=1
            all+=1
        print(correct/all)
    

### Label Encoding

In [ ]:
model = LSTM(input_dim=7, hidden_dim=10, num_layers=2, output_dim=1)
model.train()
optimizer = optim.Adadelta(model.parameters(), lr = .05)
criterion = torch.nn.MSELoss()
epochs = 50
for epoch in range(epochs):
    correct = 0
    all = 0
    for _, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        # print(y)
        pred_y = model(X)
        
        loss = criterion(pred_y, y)
        
        loss.backward()
        
        optimizer.step()
        #print(pred_y)
        print(f'-----{epoch}------')
        for k, i in enumerate(pred_y):
            if np.argmax(i.detach().numpy())-1 == y[k]:
                print(y)
                correct +=1
            all += 1
        print(correct/all)